In [1]:
from neo4j import GraphDatabase, basic_auth

driver = GraphDatabase.driver("bolt://localhost:7687", auth=basic_auth("neo4j", "12345678"))
session = driver.session()

In [2]:
#Q1
query1 = """
MATCH (p:Person)-[:PRODUCED]->(m:Movie)
WHERE m.title = 'When Harry Met Sally'
RETURN p.name AS producerNAME
"""
producer = driver.session().run(query1)
for result in producer:
    print(result['producerNAME'])

Rob Reiner
Nora Ephron


In [3]:
#Q2
query2 = """
MATCH (d:Person)-[:DIRECTED]->(m:Movie)
WITH d, COUNT(m) AS MoviesDirected
WHERE MoviesDirected > 2
RETURN d.name AS DirectorName, MoviesDirected
ORDER BY MoviesDirected DESC
"""

count = driver.session().run(query2)
for result in count:
    print(f"Director: {result['DirectorName']}, Movies Directed: {result['MoviesDirected']}")

Director: Larry Wachowski, Movies Directed: 5
Director: Andy Wachowski, Movies Directed: 5
Director: Rob Reiner, Movies Directed: 3
Director: Ron Howard, Movies Directed: 3


In [4]:
#Q3
query3 = """
MATCH (a:Person)-[:ACTED_IN]->(m:Movie)
WITH a, COUNT(m) AS NumberOfMovies
WHERE NumberOfMovies >= 5
RETURN a.name AS ActorName, NumberOfMovies
ORDER BY NumberOfMovies DESC
"""
actors = driver.session().run(query3)
for result in actors:
    print(f"Actor: {result['ActorName']}, Movies Acted: {result['NumberOfMovies']}")

Actor: Tom Hanks, Movies Acted: 12
Actor: Keanu Reeves, Movies Acted: 7
Actor: Hugo Weaving, Movies Acted: 5
Actor: Jack Nicholson, Movies Acted: 5
Actor: Meg Ryan, Movies Acted: 5


In [5]:
#Q4
query4 = """
MATCH (p:Person)-[:PRODUCED]->(m:Movie),
      (p)-[:WROTE]->(m),
      (p)-[:DIRECTED]->(m)
RETURN p.name AS PersonName, m.title AS MovieTitle
"""
triple_roles = driver.session().run(query4)
for result in triple_roles:
    print(f"Person: {result['PersonName']}, Movie: {result['MovieTitle']}")

Person: Cameron Crowe, Movie: Jerry Maguire
Person: Nancy Meyers, Movie: Something's Gotta Give


In [6]:
#Q5
query5 = """
MATCH (hanks:Person {name: "Tom Hanks"})-[:ACTED_IN]->(m1:Movie)<-[:ACTED_IN]-(actor1:Person),
      (actor1)-[:ACTED_IN]->(m2:Movie)<-[:ACTED_IN]-(actor2:Person),
      (actor2)-[:ACTED_IN]->(m3:Movie)<-[:ACTED_IN]-(actor3:Person)
WHERE NOT (hanks)-[:ACTED_IN]->()<-[:ACTED_IN]-(actor3) AND hanks <> actor3
RETURN DISTINCT actor3.name AS actorName

"""

with driver.session() as session:
    actors_3_degrees = session.run(query5)
    for result in actors_3_degrees:
        print(result["actorName"])

Zach Grenier
Danny DeVito
John C. Reilly
J.T. Walsh
Keanu Reeves
Diane Keaton
James Marshall
Kevin Pollak
Aaron Sorkin
Cuba Gooding Jr.
Christopher Guest
Noah Wyle
Kiefer Sutherland
Demi Moore
Tom Cruise
Robin Williams
Max von Sydow
Werner Herzog
Annabella Sciorra
Jerry O'Connell
Jay Mohr
Jonathan Lipnicki
Renee Zellweger
Kelly Preston
Regina King
Jack Nicholson
Gene Hackman
River Phoenix
Wil Wheaton
John Cusack
Corey Feldman
Marshall Bell
Val Kilmer
Tom Skerritt
Kelly McGillis
Anthony Edwards
Oliver Platt
Frank Langella
Michael Sheen
Ethan Hawke
Rick Yune
Christian Bale
Clint Eastwood
Richard Harris
Brooke Langton
Orlando Jones
Ice-T
Dina Meyer
Takeshi Kitano
Laurence Fishburne
Carrie-Anne Moss
Emil Eifrem
Susan Sarandon
John Goodman
Christina Ricci
Matthew Fox
Rain
Emile Hirsch
Naomie Harris
Al Pacino
Carrie Fisher
Billy Crystal
Bruno Kirby
Ben Miles


In [7]:
#Q6
query6 = """
MATCH (p:Person {name: "Lana Wachowski"})
SET p.name = "Larry Wachowski"
RETURN p.name AS newName
"""
updated_name = driver.session().run(query6)

In [8]:
#Q7
query7 = """
MATCH (movie:Movie)
WHERE toLower(movie.summary) CONTAINS 'fun'
CREATE (section:Section {name: 'Fun Movies'})
MERGE (section)-[:INCLUDES]->(movie)
RETURN section, movie
"""
fun_movies = driver.session().run(query7)

In [9]:
# #Q8
query8 = """
MATCH (m:Movie)
RETURN m.released AS ReleaseYear, COUNT(*) AS MovieCount
ORDER BY ReleaseYear
"""
movies_by_year = driver.session().run(query8)
for result in movies_by_year:
    print(f"Release Year: {result['ReleaseYear']}, Movie Count: {result['MovieCount']}")

Release Year: 1975, Movie Count: 1
Release Year: 1986, Movie Count: 2
Release Year: 1990, Movie Count: 1
Release Year: 1992, Movie Count: 4
Release Year: 1993, Movie Count: 1
Release Year: 1995, Movie Count: 2
Release Year: 1996, Movie Count: 3
Release Year: 1997, Movie Count: 2
Release Year: 1998, Movie Count: 3
Release Year: 1999, Movie Count: 4
Release Year: 2000, Movie Count: 3
Release Year: 2003, Movie Count: 3
Release Year: 2004, Movie Count: 1
Release Year: 2006, Movie Count: 3
Release Year: 2007, Movie Count: 1
Release Year: 2008, Movie Count: 2
Release Year: 2009, Movie Count: 1
Release Year: 2012, Movie Count: 1


In [10]:
#Q9
query9 = """
MATCH (p:Person)-[r:REVIEWED]->(m:Movie)
WHERE r.rating >= 90
RETURN m.title AS Movie,r.rating as movieRating
"""
high_rating_movies = driver.session().run(query9)
for result in high_rating_movies:
    print(f"Movie: {result['Movie']}, Rating: {result['movieRating']}")

Movie: Cloud Atlas, Rating: 95
Movie: The Replacements, Rating: 100


In [11]:
#Q10
query10 = """
MATCH (p:Person)-[:PRODUCED]->(m:Movie)
WITH p.name AS Producer, COUNT(m) AS MovieCount
RETURN Producer, MovieCount
ORDER BY MovieCount DESC
"""
top_producers = driver.session().run(query10)
for result in top_producers:
    print(f"Producer: {result['Producer']}, Movies Produced: {result['MovieCount']}")

Producer: Joel Silver, Movies Produced: 6
Producer: Andy Wachowski, Movies Produced: 2
Producer: Larry Wachowski, Movies Produced: 2
Producer: Cameron Crowe, Movies Produced: 1
Producer: Rob Reiner, Movies Produced: 1
Producer: Nora Ephron, Movies Produced: 1
Producer: Nancy Meyers, Movies Produced: 1
